# Train chatbot
<hr>

In [ ]:
"""
python=3.7
did pip
tensorflow==2.3.1
nltk==3.5
colorama==0.4.3
numpy==1.18.5
scikit_learn==0.23.2
Flask==1.1.2

"""

In [1]:
# Import

import json 
import numpy as np 

import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [3]:
# Open json file
with open('../data/intents.json') as file:
    data = json.load(file)

# Var define
training_sentences = []
training_labels = []
labels = []
responses = []

# loop to load data 
for intent in data['intents']:
    for pattern in intent['patterns']:
        #
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    #
    responses.append(intent['responses'])

    # creates labels list
    if intent['tag'] not in labels:
        labels.append(intent['tag'])


num_classes = len(labels)

In [4]:

# Transform Target Labels 

# define model 
lbl_encoder = LabelEncoder()
# fit model 
lbl_encoder.fit(training_labels)
# transform model 
training_labels = lbl_encoder.transform(training_labels)



 We can also add “oov_token” which is a value for “out of token” to deal with out of vocabulary words(tokens) at inference time.

In [6]:
# 

# Var define
vocab_size = 1000
embedding_dim = 16
max_len = 20 
oov_token = '<00V>'

# declare tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
# fit tokenier
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
# creates equal sizes between the text sequences
padded_sequences = pad_sequences(sequences, truncating = 'post', maxlen = max_len)


In [8]:
# Define NN Model

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation = 'softmax'))


In [9]:
# Compile model

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


In [10]:
# Model Summory 

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________________________________________

In [11]:
# Train Model 

epochs = 500
history = model.fit(padded_sequences, np.array(training_labels),  epochs=epochs)

Epoch 1/500
2/2 [==============================] - 1s 6ms/step - loss: 2.0822 - accuracy: 0.0909
Epoch 2/500
2/2 [==============================] - 0s 13ms/step - loss: 2.0798 - accuracy: 0.0909
Epoch 3/500
2/2 [==============================] - 0s 10ms/step - loss: 2.0787 - accuracy: 0.2424
Epoch 4/500
2/2 [==============================] - 0s 6ms/step - loss: 2.0778 - accuracy: 0.1515
Epoch 5/500
2/2 [==============================] - 0s 5ms/step - loss: 2.0771 - accuracy: 0.1515
Epoch 6/500
2/2 [==============================] - 0s 4ms/step - loss: 2.0762 - accuracy: 0.1515
Epoch 7/500
2/2 [==============================] - 0s 4ms/step - loss: 2.0753 - accuracy: 0.1515
Epoch 8/500
2/2 [==============================] - 0s 5ms/step - loss: 2.0745 - accuracy: 0.1515
Epoch 9/500
2/2 [==============================] - 0s 12ms/step - loss: 2.0738 - accuracy: 0.1515
Epoch 10/500
2/2 [==============================] - 0s 6ms/step - loss: 2.0732 - accuracy: 0.1515
Epoch 11/500
2/2 [========

# Making a Pickle
<hr>

In [13]:
# save the trained model 
model.save('../assets/chat_model')

# save the fitted Tokenizer
with open('../assets/tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# to save the fittend label encoder
with open('../assets/label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)


    

INFO:tensorflow:Assets written to: ../assets/chat_model/assets
